# Introduction à la programmation par contrainte

La **programmation par contrainte** (*Constraint Programming*) est une approche de résolution de problèmes où
l’on définit un ensemble de **variables**, chacune ayant un **domaine** de valeurs possibles, ainsi qu’un
ensemble de **contraintes** exprimant des relations entre ces variables. L’objectif est de trouver une
affectation (choisir une valeur pour chaque variable) satisfaisant *toutes* les contraintes.

Les **CSP** (Constraint Satisfaction Problems) apparaissent dans de nombreux domaines :
- Sudoku,
- Coloration de graphes (carte ou régions),
- Problèmes d’emplois du temps,
- Problèmes de planification, etc.

Dans ce notebook, nous allons :
- Comprendre la structure d’un CSP (variables, domaines, contraintes).
- Implémenter un solveur simple par **backtracking** (recherche en profondeur avec retour arrière).
- Illustrer ces concepts sur un petit problème de **coloration de graphe**.


In [ ]:
class CSP:
    """
    Classe très simplifiée de type 'Constraint Satisfaction Problem'.

    - variables : liste de noms de variables
    - domains : dict qui associe à chaque variable la liste (ou ensemble) de valeurs possibles
    - neighbors : dict qui associe à chaque variable la liste de ses variables voisines (qui partagent une contrainte)
    - constraint_func : fonction (var1, val1, var2, val2) -> bool (True si pas de conflit)

    n_assigns : compteur d'assignations tentées (pour info)
    """

    def __init__(self, variables, domains, neighbors, constraint_func):
        self.variables = variables
        self.domains = domains
        self.neighbors = neighbors
        self.constraint_func = constraint_func
        self.n_assigns = 0  # compteur

    def consistent(self, var, val, assignment):
        """
        Vérifie la cohérence de (var=val) avec l'affectation partielle 'assignment',
        en respectant la contrainte (constraint_func) pour chaque voisin déjà affecté.
        """
        for other_var in assignment:
            if other_var in self.neighbors[var]:
                other_val = assignment[other_var]
                # S'il y a conflit avec un voisin, on retourne False
                if not self.constraint_func(var, val, other_var, other_val):
                    return False
        return True


# Exemple : Coloration de graphe minimal

Imaginons un mini-problème où nous devons colorer 4 régions (A, B, C et D), avec 3 couleurs possibles
(Rouge, Vert, Bleu). Deux régions voisines ne doivent pas avoir la même couleur.

- Variables : `["A", "B", "C", "D"]`
- Domaines : par exemple `{"A": ["R", "V", "B"], "B": ...}` etc.
- Voisins : A voisine de B et C, B voisine de A et C, C voisine de A, B, D, et D voisine de C.
- La fonction de contrainte : deux régions voisines ne peuvent pas avoir la même couleur.


In [ ]:
# 1. Variables du problème
vars_graph = ["A", "B", "C", "D"]

# 2. Domaines (chaque variable peut prendre les couleurs R, V, B)
dom_graph = {
    "A": ["R", "V", "B"],
    "B": ["R", "V", "B"],
    "C": ["R", "V", "B"],
    "D": ["R", "V", "B"]
}

# 3. Liste des voisins
neighbors_graph = {
    "A": ["B", "C"],
    "B": ["A", "C"],
    "C": ["A", "B", "D"],
    "D": ["C"]
}

# 4. Contrainte : deux régions voisines ne peuvent pas avoir la même couleur
def different_colors_constraint(v1, val1, v2, val2):
    return val1 != val2

# Construction du CSP
graph_csp = CSP(vars_graph, dom_graph, neighbors_graph, different_colors_constraint)


# Backtracking : principe

L’algorithme de backtracking procède ainsi :
1. On choisit une variable non encore affectée.
2. On parcourt les valeurs possibles de son domaine.
3. Pour chaque valeur, on vérifie si elle est cohérente avec l’affectation courante (pas de conflit).
4. Si c’est cohérent, on assigne la variable et on passe à la variable suivante.
5. Si on ne trouve pas de valeur valable, on “revient en arrière” (backtrack).

C’est un algorithme de base, qui peut être beaucoup amélioré par des heuristiques (MRV, LCV, etc.).


In [ ]:
def backtracking_search(csp, assignment=None):
    # assignment est un dict var -> val
    if assignment is None:
        assignment = {}

    # Si toutes les variables sont affectées, on a une solution
    if len(assignment) == len(csp.variables):
        return assignment

    # Choisir une variable pas encore affectée
    unassigned_vars = [v for v in csp.variables if v not in assignment]
    var = unassigned_vars[0]  # simple : on prend la première

    # Parcourir les valeurs possibles pour cette variable
    for val in csp.domains[var]:
        csp.n_assigns += 1  # compteur de tentatives
        if csp.consistent(var, val, assignment):
            # On assigne et on continue
            assignment[var] = val
            result = backtracking_search(csp, assignment)
            if result is not None:
                return result
            # Sinon, on retire l'assignation et on essaie une autre valeur
            del assignment[var]

    # Pas de solution sur ce chemin
    return None


In [ ]:
solution = backtracking_search(graph_csp)

print("Solution trouvée :", solution)
print("Nombre d'assignations tentées :", graph_csp.n_assigns)

# Vérifions que toutes les paires voisines sont de couleurs différentes
if solution:
    ok = True
    for var in graph_csp.variables:
        val = solution[var]
        for voisin in graph_csp.neighbors[var]:
            if solution[voisin] == val:
                ok = False
    print("Vérification -> la solution est valide." if ok else "Oups, solution invalide.")


## Aller plus loin : améliorations et heuristiques

Le backtracking \"naïf\" présenté jusqu'ici fonctionne déjà correctement sur de petits problèmes,  
mais il peut s'avérer très lent lorsque le problème grandit.  

Heureusement, plusieurs améliorations/heuristiques sont possibles :

1. **Heuristique de sélection de variable** :  
   - **MRV (Minimum Remaining Values)** : on choisit d'abord la variable qui dispose du **plus petit** domaine restant.
   - **Degré (Degree Heuristic)** : on choisit la variable la plus \"contraignante\", c.-à-d. celle qui a le plus de voisins non encore affectés.

2. **Heuristique de tri des valeurs** :  
   - **LCV (Least Constraining Value)** : on essaye d'abord la valeur qui restreint le moins les autres variables.

3. **Propagation de contraintes** :  
   - **Forward Checking** : à chaque affectation, on \"prune\" (restreint) les valeurs impossibles des domaines des variables voisines.
   - **Arc Consistency (AC-3)** : on vérifie la cohérence des arcs de contraintes et on élimine à l'avance certaines valeurs.

Nous allons implémenter **MRV** et **LCV**, puis un petit mécanisme de **forward checking** pour illustrer comment aller plus loin.


In [ ]:
def select_unassigned_variable_mrv(assignment, csp):
    """
    Heuristique MRV : on choisit la variable non assignée dont le domaine possible
    est le plus restreint (i.e. le moins de valeurs viables).
    """
    unassigned = [v for v in csp.variables if v not in assignment]
    # Pour chaque variable, on compte le nombre de valeurs qui restent possibles
    # si on la testait en l'état, compte tenu de 'assignment'.
    # On calcule ce 'reste' en vérifiant la cohérence de (var=val) pour chaque val du domaine.
    
    def possible_values_count(var):
        count_ok = 0
        for val in csp.domains[var]:
            if csp.consistent(var, val, assignment):
                count_ok += 1
        return count_ok
    
    # On renvoie la variable qui a le moins de valeurs possibles (argmin)
    # en cas d'égalité, Python peut en rendre une au hasard parmi les ex aequo.
    return min(unassigned, key=possible_values_count)


In [ ]:
def order_domain_values_lcv(var, assignment, csp):
    """
    Heuristique LCV : on trie les valeurs de var de celle qui 'contraint' le moins
    les autres, à celle qui les contraint le plus.
    
    Concrètement, on compte le nombre de conflits que (var=val) entraînerait chez les voisins,
    et on classe les valeurs par ordre croissant de 'conflit'.
    """
    def conflict_count(val):
        # Compte combien de valeurs sont rendues impossibles dans les voisins
        nb_conflicts = 0
        for voisin in csp.neighbors[var]:
            if voisin not in assignment:
                # Pour chaque valeur possible du voisin, si la contrainte échoue, c'est un \"conflit potentiel\"
                for vval in csp.domains[voisin]:
                    # On simule la contrainte (var=val) et (voisin=vval)
                    # s'il y a un conflit, on incrémente
                    if not csp.constraint_func(var, val, voisin, vval):
                        nb_conflicts += 1
        return nb_conflicts
    
    # On trie toutes les valeurs du domaine en fonction du nb de \"conflits\" provoqués
    return sorted(csp.domains[var], key=conflict_count)


In [ ]:
def forward_checking(csp, var, val, assignment):
    """
    Petit mécanisme de forward checking :
    - on va éliminer (prune) les valeurs impossibles dans les domaines des voisins
      une fois qu'on a choisi var=val.
    - on renvoie True si on ne détecte pas d'incohérence, False sinon.
    
    NB: on va faire un \"rollback\" si on détecte une incohérence, i.e.
    si un voisin n'a plus de valeur possible.
    
    Pour la version simple, on modifie directement csp.domains. 
    On pourrait stocker les modifs et les rétablir au besoin (removals...).
    """
    for voisin in csp.neighbors[var]:
        if voisin not in assignment:
            # Pour chaque valeur potentielle du voisin, on teste la contrainte
            to_remove = []
            for vval in csp.domains[voisin]:
                # S'il y a conflit direct (i.e. incompatible avec var=val),
                # on enlève vval du domaine du voisin
                if not csp.constraint_func(var, val, voisin, vval):
                    to_remove.append(vval)
            
            # On retire ces valeurs du domaine
            for valrem in to_remove:
                csp.domains[voisin].remove(valrem)
            
            # S'il ne reste plus de valeur, c'est échec
            if len(csp.domains[voisin]) == 0:
                return False
    return True


In [ ]:
def backtracking_search_enhanced(csp, assignment=None):
    """
    Backtracking amélioré avec MRV, LCV et Forward Checking.
    """
    if assignment is None:
        assignment = {}
    
    # Check if we are done
    if len(assignment) == len(csp.variables):
        return assignment
    
    # MRV : on sélectionne la variable avec le plus petit domaine viable
    var = select_unassigned_variable_mrv(assignment, csp)
    
    # On essaye les valeurs dans l'ordre LCV
    ordered_values = order_domain_values_lcv(var, assignment, csp)
    
    # Pour sauvegarder l'état du domaine avant modif
    saved_domains = {v: list(csp.domains[v]) for v in csp.variables}
    
    for val in ordered_values:
        csp.n_assigns += 1
        
        if csp.consistent(var, val, assignment):
            assignment[var] = val
            # On tente un forward checking
            if forward_checking(csp, var, val, assignment):
                # Recursion
                result = backtracking_search_enhanced(csp, assignment)
                if result is not None:
                    return result
            
            # Sinon, ou si la suite échoue, on \"rollback\"
            # on rétablit l'état des domaines avant d'essayer la valeur suivante
            for v in csp.variables:
                csp.domains[v] = list(saved_domains[v])
            
            del assignment[var]
    
    return None


## Démonstration des heuristiques sur le même mini-problème de coloration

On va réutiliser exactement le même CSP de \"coloration de graphe\" (A, B, C, D)  
et voir comment le solveur amélioré se comporte.  


In [ ]:
# On réinitialise un CSP identique
vars_graph = ["A", "B", "C", "D"]
dom_graph = {
    "A": ["R", "V", "B"],
    "B": ["R", "V", "B"],
    "C": ["R", "V", "B"],
    "D": ["R", "V", "B"]
}
neighbors_graph = {
    "A": ["B", "C"],
    "B": ["A", "C"],
    "C": ["A", "B", "D"],
    "D": ["C"]
}
def different_colors_constraint(v1, val1, v2, val2):
    return val1 != val2

graph_csp2 = CSP(vars_graph, dom_graph, neighbors_graph, different_colors_constraint)

solution2 = backtracking_search_enhanced(graph_csp2)

print("Solution trouvée (avec heuristiques) :", solution2)
print("Nombre d'assignations tentées :", graph_csp2.n_assigns)


## Arc Consistency (AC-3)

Pour aller plus loin dans la propagation de contraintes, on peut utiliser **AC-3** (Arc Consistency Algorithm 3).  
Le principe : pour chaque arc \((X, Y)\), on vérifie que chaque valeur de \(X\) est « supportée » par au moins une valeur de \(Y\). Si ce n’est pas le cas, on élimine cette valeur du domaine de \(X\). On répète jusqu’à stabilité ou jusqu’à ce qu’un domaine devienne vide (inconsistance).

Voici une implémentation simple de AC-3 :


In [ ]:
def AC3(csp):
    """
    Rend le CSP arc-consistent en modifiant csp.domains.
    Retourne True si réussi, ou False si un domaine devient vide.
    """
    # Construire la file de tous les arcs (X, Y)
    queue = []
    for X in csp.variables:
        for Y in csp.neighbors[X]:
            queue.append((X, Y))

    while queue:
        (X, Y) = queue.pop(0)
        if revise(csp, X, Y):
            if len(csp.domains[X]) == 0:
                return False  # Inconsistance
            # Ré-ajouter les arcs (Z, X) pour tous les Z voisins de X
            for Z in csp.neighbors[X]:
                if Z != Y:
                    queue.append((Z, X))
    return True

def revise(csp, X, Y):
    """
    retire de csp.domains[X] les valeurs qui ne sont supportées
    par aucune valeur dans csp.domains[Y].
    Retourne True si on a retiré au moins une valeur.
    """
    revised = False
    to_remove = []
    for xval in csp.domains[X]:
        # Cherche une valeur yval qui satisfasse la contrainte
        supported = False
        for yval in csp.domains[Y]:
            if csp.constraint_func(X, xval, Y, yval):
                supported = True
                break
        if not supported:
            to_remove.append(xval)

    for val in to_remove:
        csp.domains[X].remove(val)
    if to_remove:
        revised = True
    return revised


### Exemple d’utilisation d’AC-3 sur le même CSP de coloration

On peut d’abord rendre le CSP arc-consistent, puis lancer le backtracking.


In [ ]:
# On reprend le CSP 'graph_csp' ou 'graph_csp2', peu importe :
# (Assure-toi seulement qu'il n'a pas déjà été modifié par un précédent forward_checking.)

# Applique AC-3
arc_ok = AC3(graph_csp2)
print("AC-3 terminé. Est-ce cohérent ? ", arc_ok)
print("Domaines restants après AC-3 :")
for v in graph_csp2.variables:
    print(f" {v} : {graph_csp2.domains[v]}")

# Ensuite on relance un backtracking par exemple
sol_ac3 = backtracking_search(graph_csp2)
print("Solution après AC-3 :", sol_ac3)


## Autre exemple : le problème des n reines

Le problème des n reines consiste à placer n reines sur un échiquier \(n\times n\) pour qu’aucune ne se menace :

- **Variables** : un entier par colonne (de 0 à n−1).
- **Domaines** : chaque variable peut prendre une ligne de 0 à n−1.
- **Contraintes** : deux reines ne partagent pas la même ligne ni la même diagonale.

On peut définir un CSP `n_queens_csp(n)` :


In [ ]:
def n_queens_csp(n):
    """
    Construit un CSP pour le problème des n reines.
    variables : [0..n-1] (chaque colonne)
    domaines : [0..n-1] pour chacune (position de la reine dans la colonne)
    contrainte : pas même ligne, ni diagonale.
    """
    variables = list(range(n))
    domains = {col: list(range(n)) for col in variables}
    neighbors = {col: [c for c in variables if c != col] for col in variables}

    def queens_constraint(c1, r1, c2, r2):
        if r1 == r2:
            return False  # même ligne
        if abs(c1 - c2) == abs(r1 - r2):
            return False  # même diagonale
        return True

    return CSP(variables, domains, neighbors, queens_constraint)

def print_queens_solution(solution):
    """
    Affiche visuellement la solution (un dict col->row).
    """
    if not solution:
        print("Aucune solution trouvée.")
        return
    n = len(solution)
    for row in range(n):
        row_str = ""
        for col in range(n):
            if solution.get(col, None) == row:
                row_str += " Q "
            else:
                row_str += " . "
        print(row_str)

# Testons pour n=8
csp_8_queens = n_queens_csp(8)
print("Application de AC-3 sur n=8 reines...")
AC3(csp_8_queens)

solution_8 = backtracking_search(csp_8_queens)
print("Solution 8 reines (col -> row) :", solution_8)
print("Nombre d'assignations :", csp_8_queens.n_assigns)
print("Affichage de l'échiquier :")
print_queens_solution(solution_8)


## Méthode supplémentaire : Min-Conflicts

Au-delà du backtracking et de l’AC-3, il existe des approches **stochastiques** pour résoudre un CSP, comme l’algorithme **Min-Conflicts**. 

L’idée générale est :
1. Affecter aléatoirement une valeur à chaque variable (solution complète, mais potentiellement incohérente).
2. Tant qu’il y a conflit, on sélectionne une variable en conflit **au hasard**, puis on lui attribue la valeur (parmi son domaine) qui minimise le nombre total de conflits.
3. Répéter jusqu’à trouver une solution ou jusqu’à un nombre limite d’itérations.

Cet algorithme fonctionne bien sur certains CSP de grande taille (comme le problème des n reines). Il n’est pas garanti de toujours trouver une solution s’il y en a une, mais il fonctionne souvent très efficacement en pratique.


In [ ]:
import random

def min_conflicts(csp, max_steps=10000):
    """
    Résout un CSP par l’algorithme Min-Conflicts (recherche locale).
    - On commence avec une affectation complète (aléatoire ou heuristique).
    - À chaque étape, on choisit une variable conflictuelle
      et on lui assigne la valeur qui minimise le nombre de conflits.
    - On arrête si on trouve une solution ou si on dépasse max_steps.
    """
    # Affectation initiale aléatoire
    current = {var: random.choice(csp.domains[var]) for var in csp.variables}
    csp.n_assigns = 0

    def conflicts(var, val):
        """Compte les conflits pour var=val dans l'affectation current."""
        count = 0
        for neighbor in csp.neighbors[var]:
            if neighbor in current:
                neighbor_val = current[neighbor]
                if not csp.constraint_func(var, val, neighbor, neighbor_val):
                    count += 1
        return count

    for step in range(max_steps):
        # Variables actuellement en conflit
        conflicted = [v for v in csp.variables if conflicts(v, current[v]) > 0]

        if not conflicted:
            # Solution trouvée
            return current

        # Choisit aléatoirement une variable conflictuelle
        var = random.choice(conflicted)

        # Sélectionne la valeur minimisant les conflits (avec tie-breaking aléatoire)
        conflict_counts = [(val, conflicts(var, val)) for val in csp.domains[var]]
        min_conflict = min(count for val, count in conflict_counts)
        best_vals = [val for val, count in conflict_counts if count == min_conflict]

        # Assigne aléatoirement parmi les meilleures valeurs
        current[var] = random.choice(best_vals)
        csp.n_assigns += 1

    return None  # Pas trouvé en max_steps


# Démonstration : n reines avec min_conflicts
n = 8
csp_8_mc = n_queens_csp(n)

solution_mc = min_conflicts(csp_8_mc, max_steps=100000)
print(f"Min-Conflicts pour {n} reines -> Solution:", solution_mc)
print("Nombre d'assignations tentées (compteur interne) :", csp_8_mc.n_assigns)

if solution_mc:
    print("Affichage de la solution :")
    print_queens_solution(solution_mc)
else:
    print("Pas de solution trouvée (max_steps atteint).")


## Conclusion

Dans ce notebook, nous avons :

- Défini une classe `CSP` simple (variables, domaines, voisins, fonction de contrainte).
- Codé un solveur **backtracking** basique, puis un solveur **amélioré** (MRV, LCV, forward checking).
- Ajouté un algorithme de propagation **AC-3** pour rendre le CSP arc-consistent avant ou pendant la recherche.
- Illustré le tout sur un mini-exemple de **coloration de graphe**, puis sur le problème des **n reines**.

Ces techniques (backtracking, heuristiques, propagation) constituent les fondements de la programmation par contrainte. Pour des cas plus grands, on utilisera souvent des bibliothèques dédiées (Python-Constraint, OR-Tools, etc.) qui intègrent des algorithmes plus avancés et très optimisés.  
Néanmoins, ce code vous donne une base compréhensible et modulable pour expérimenter sur de petits CSP ! 


## Références et bibliothèques utiles

- **[Python-Constraint](https://labix.org/python-constraint)** : une bibliothèque Python proposant différents solveurs CSP (backtracking, min-conflicts, etc.).
- **[OR-Tools (Google)](https://developers.google.com/optimization)** : framework de résolution (CSP, MILP, SAT, VRP...) très performant, en Python/C++/Java.
- **[Choco Solver (Java)](https://choco-solver.org/)** : autre bibliothèque de programmation par contrainte très complète.
- Section *Constraint Satisfaction Problems* dans le livre *Artificial Intelligence: A Modern Approach* (Russell & Norvig, Chapter 6) pour un exposé plus détaillé.
